In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from skimage.transform import hough_line, hough_line_peaks

## Поиск прямых преобразованием Хафа

In [ ]:
CANNY_THRESH1 = 50
CANNY_THRESH2 = 150
HOUGH_THRESHOLD = 70
HOUGH_MIN_LINE_LENGTH = 200
HOUGH_MAX_LINE_GAP = 100

USE_BLUR = True
blur_ksize = 5
img_save = True

image_paths = ['images/barcode2.png', 'images/pedestrian.png', 'images/aviasales.png']

def detect_lines_via_houghP(edges):
    linesP = cv2.HoughLinesP(edges,
                             rho=1,
                             theta=np.pi / 180,
                             threshold=HOUGH_THRESHOLD,
                             minLineLength=HOUGH_MIN_LINE_LENGTH,
                             maxLineGap=HOUGH_MAX_LINE_GAP)
    return linesP


def measure_lines(lines):
    if lines is None or len(lines) == 0:
        return 0, 0, 0

    lengths = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        lengths.append(dist)
    return len(lengths), min(lengths), max(lengths)


def draw_lines_on_image(image_rgb, lines, color=(0, 255, 0), thickness=2):
    out = image_rgb.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(out, (x1, y1), (x2, y2), color, thickness)
            cv2.circle(out, (x1, y1), 4, (255, 0, 0), -1)
            cv2.circle(out, (x2, y2), 4, (0, 255, 255), -1)
    return out


for i, img_path in enumerate(image_paths, start=1):
    bgr = cv2.imread(img_path)
    if bgr is None:
        print(f'Не удалось открыть {img_path}')
        continue

    h, w = bgr.shape[:2]
    dpi = 100

    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    plt.imshow(rgb)
    plt.axis('off')
    plt.show()

    linesP = detect_lines_via_houghP(gray)
    count_lines, min_len, max_len = measure_lines(linesP)

    hspace, angles, dists = hough_line(gray)
    o_img_with_lines = draw_lines_on_image(rgb, linesP, color=(255, 0, 0), thickness=2)

    print(f'Оригинальное изображение {i}:')
    print(f'Найдено прямых: {count_lines}')
    if count_lines > 0:
        print(f'Длина самой короткой: {min_len:.2f}')
        print(f'Длина самой длинной:  {max_len:.2f}')
    else:
        print('  Прямых не найдено.')

    plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    plt.imshow(o_img_with_lines)
    plt.axis('off')
    if img_save:
        plt.savefig(f"images/hough_lines/{i}_orig_hough_lines.png")
    plt.show()

    plt.figure(figsize=(6, 5))
    plt.imshow(np.log(1 + hspace),
               extent=(np.rad2deg(angles[0]), np.rad2deg(angles[-1]), dists[-1], dists[0]),
               aspect='auto',
               cmap='jet')
    plt.xlabel('Угол')
    plt.ylabel('Расстояние')
    if img_save:
        plt.savefig(f"images/hough_lines/{i}_orig_accum.png")
    plt.show()

    if USE_BLUR and blur_ksize > 0:
        gray = cv2.GaussianBlur(gray, (blur_ksize, blur_ksize), 0)

    edges = cv2.Canny(gray, CANNY_THRESH1, CANNY_THRESH2)

    linesP = detect_lines_via_houghP(edges)
    count_lines, min_len, max_len = measure_lines(linesP)

    hspace, angles, dists = hough_line(edges)
    p_img_with_lines = draw_lines_on_image(rgb, linesP, color=(255, 0, 0), thickness=2)

    print(f'Преобразованное изображение {i}:')
    print(f'Найдено прямых: {count_lines}')
    if count_lines > 0:
        print(f'Длина самой короткой: {min_len:.2f}')
        print(f'Длина самой длинной:  {max_len:.2f}')
    else:
        print('  Прямых не найдено.')

    plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    plt.imshow(edges, cmap='gray')
    plt.axis('off')
    if img_save:
        plt.savefig(f"images/hough_lines/{i}_proc_canny.png")
    plt.show()

    plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    plt.imshow(p_img_with_lines)
    plt.axis('off')
    if img_save:
        plt.savefig(f"images/hough_lines/{i}_proc_hough_lines.png")
    plt.show()

    plt.figure(figsize=(6, 5))
    plt.imshow(np.log(1 + hspace),
               extent=(np.rad2deg(angles[0]), np.rad2deg(angles[-1]), dists[-1], dists[0]),
               aspect='auto',
               cmap='jet')
    plt.xlabel('Угол')
    plt.ylabel('Расстояние')
    if img_save:
        plt.savefig(f"images/hough_lines/{i}_proc_accum.png")
    plt.show()

## Поиск окружностей преобразованием Хафа


In [ ]:
USE_BLUR       = True
BLUR_KSIZE     = 5

CANNY_THRESH1  = 200
CANNY_THRESH2  = 450

FIXED_RADIUS   = [170, 50, 200]
R = [(100, 700), (50, 100), (60, 600)]
CIRCLE_DP = 1
CIRCLE_MIN_DIST = [30, 5, 50]
CIRCLE_PARAM1   = [700, 100, 600]
CIRCLE_PARAM2   = [100, 50, 60]

IMG_SAVE = True

image_paths = [
    'images/cola.png',
    'images/money.png',
    'images/wheel.png'
]

def draw_circles(img, circles, color=(255,0,0), thickness=2, mark_center=True):
    out = img.copy()
    if circles is not None:
        for x,y,r in np.round(circles[0]).astype(int):
            cv2.circle(out, (x,y), r, color, thickness)
            if mark_center:
                cv2.circle(out, (x,y), 3, (0,255,0), -1)
    return out

for idx, path in enumerate(image_paths, start=1):
    bgr = cv2.imread(path)
    if bgr is None:
        print(f'Не удалось открыть {path}')
        continue

    h, w = bgr.shape[:2]
    dpi = 100

    rgb  = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    if USE_BLUR:
        gray_blur = cv2.GaussianBlur(gray, (BLUR_KSIZE,BLUR_KSIZE), 0)
    else:
        gray_blur = gray

    circles_fixed  = cv2.HoughCircles(
        gray_blur,
        cv2.HOUGH_GRADIENT,
        dp=CIRCLE_DP,
        minDist=CIRCLE_MIN_DIST[idx-1],
        param1=CIRCLE_PARAM1[idx-1],
        param2=CIRCLE_PARAM2[idx-1],
        minRadius=FIXED_RADIUS[idx-1],
        maxRadius=FIXED_RADIUS[idx-1]
    )
    circles_range  = cv2.HoughCircles(
        gray_blur,
        cv2.HOUGH_GRADIENT,
        dp=CIRCLE_DP,
        minDist=CIRCLE_MIN_DIST[idx-1],
        param1=CIRCLE_PARAM1[idx-1],
        param2=CIRCLE_PARAM2[idx-1],
        minRadius=R[idx-1][0],
        maxRadius=R[idx-1][1]
    )

    print(f'=== Изображение {idx}: {path} ===')
    print(f'Оригинал → фикс. радиус {FIXED_RADIUS[idx-1]}:',
          0 if circles_fixed is None else len(circles_fixed[0]))
    print(f'Оригинал → диапазон радиусов [{R[idx-1][0]},{R[idx-1][1]}]:',
          0 if circles_range is None else len(circles_range[0]))

    plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    plt.imshow(rgb)
    plt.axis('off')
    plt.show()

    fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    out_fixed = draw_circles(rgb, circles_fixed, color=(255,0,0))
    plt.imshow(out_fixed)
    plt.axis('off')
    if IMG_SAVE:
        fig.savefig(f'images/hough_circles/{idx}_orig_fixed.png')
    plt.show()

    fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    out_range = draw_circles(rgb, circles_range, color=(0,0,255))
    plt.imshow(out_range)
    plt.axis('off')
    if IMG_SAVE:
        fig.savefig(f'images/hough_circles/{idx}_orig_range.png')
    plt.show()

    edges = cv2.Canny(gray, CANNY_THRESH1, CANNY_THRESH2)


    circles_fixed_p  = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=CIRCLE_DP,
        minDist=CIRCLE_MIN_DIST[idx-1],
        param1=CIRCLE_PARAM1[idx-1],
        param2=CIRCLE_PARAM2[idx-1],
        minRadius=FIXED_RADIUS[idx-1],
        maxRadius=FIXED_RADIUS[idx-1]
    )
    circles_range_p  = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=CIRCLE_DP,
        minDist=CIRCLE_MIN_DIST[idx-1],
        param1=CIRCLE_PARAM1[idx-1],
        param2=CIRCLE_PARAM2[idx-1],
        minRadius=R[idx-1][0],
        maxRadius=R[idx-1][1]
    )

    print(f'Processed → фикс. радиус {FIXED_RADIUS[idx-1]}:',
          0 if circles_fixed_p is None else len(circles_fixed_p[0]))
    print(f'Processed → диапазон радиусов [{R[idx-1][0]},{R[idx-1][1]}]:',
          0 if circles_range_p is None else len(circles_range_p[0]))

    plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    plt.imshow(edges, cmap='gray')
    plt.axis('off')
    if IMG_SAVE:
        plt.gcf().savefig(f'images/hough_circles/{idx}_proc_canny.png')
    plt.show()

    fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    out_fixed_p = draw_circles(rgb, circles_fixed_p, color=(255,0,0))
    plt.imshow(out_fixed_p)
    plt.axis('off')
    if IMG_SAVE:
        fig.savefig(f'images/hough_circles/{idx}_proc_fixed.png')
    plt.show()

    fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi)
    out_range_p = draw_circles(rgb, circles_range_p, color=(0,0,255))
    plt.imshow(out_range_p)
    plt.axis('off')
    if IMG_SAVE:
        fig.savefig(f'images/hough_circles/{idx}_proc_range.png')
    plt.show()